In [1]:
import base64
import io
import requests
from urllib.parse import urljoin
from tqdm import tqdm
import time
import yaml
from PIL import Image
import os
import cv2
import matplotlib.pyplot as plt

from abeja.datalake import Client as DatalakeClient

# set deployment_url
deployment_url = 'XXXXXXXXXXXXXXX'

# get credential information
yaml_dict = yaml.load(open('secret.yml').read(), Loader=yaml.SafeLoader)

# set credential
organization_id = yaml_dict['organization_id']
user_id = yaml_dict['user_id']
personal_access_token = yaml_dict['personal_access_token']

credential = {
    'user_id': user_id,
    'personal_access_token': personal_access_token
}

In [2]:
path = 'Dir'

for file in tqdm(os.listdir(path)):
    if os.path.isfile(os.path.join(path, file)):
        input_img = open(path + "/" + file, "rb")
        # display(input_img)
        
        res_deployment_url = requests.post(deployment_url,data=input_img,auth=(user_id, personal_access_token))

        # print(res_deployment_url)

        res_deployment_url.raise_for_status()
        labels = res_deployment_url.json()

        b64 = res_deployment_url.json()['result']
        output_img_io = io.BytesIO(base64.b64decode(b64))

        # convert black color to transparency
        img = Image.open(output_img_io).convert('RGBA')
        pixdata = img.load()

        width, height = img.size
        for y in range(height):
            for x in range(width):
                if pixdata[x, y] == (0, 0, 0, 255):
                    pixdata[x, y] = (0, 0, 0, 0)

        modified_img_io = io.BytesIO()
        savefilename = path + "/result/s_" + file
        img.save(savefilename, format='PNG')
    
        img1 = cv2.imread(path + "/" + file)
        img2 = cv2.imread(path + "/result/s_" + file)

        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

        img1 =cv2.resize(img1,(800,520))
        img2 =cv2.resize(img2,(800,520))
    
        blended = cv2.addWeighted(src1=img1,alpha=0.7,src2=img2,beta=0.3,gamma=0)
        
        plt.imshow(blended)
        plt.axis('off')
        plt.show()

100%|██████████| 2/2 [00:00<00:00, 7332.70it/s]
